# Sentiment Analysis - Transformers

In [1]:
#Import needed modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV

#Read in the data
tweets = pd.read_csv('../data/clean_tweets.csv', encoding = 'iso-8859-1')

In [2]:
#Functions to tokenize text
import string

#Replaces pos tags with lemmatize compatable tags
def pos_replace(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
#Makes list of punctuation to exclude, keeps certain symbols
punct = list(string.punctuation)
keep_punct = ['#', '?', '!', '@']
punct = [p for p in punct if p not in keep_punct]

#Used to filter rt
common_tweet_words = ['rt']

#Removes non-ASCII characters
def remove_junk(tweet):
    return ''.join([i if ord(i) < 128 else ' ' for i in tweet])
    
def tweet_tokenizer(doc):
    #Gets rid of links
    doc = re.sub(r'http\S+', '', doc)
    doc = re.sub(r'www\.[a-z]?\.?(com)+|[a-z]+\.(com)', '', doc)
    #Gets rid of #sxsw hashtag variations
    doc = re.sub(r'(?i)(#sxsw)\w*', '', doc)
    #Gets rid of conversions made during scraping
    doc = re.sub(r'{link}', '', doc)
    doc = re.sub(r'\[video\]', '', doc)
    #Gets rid of weird characters
    doc = remove_junk(doc)
    #Tokenizes using NLTK Twitter Tokenizer
    tweet_token = TweetTokenizer(strip_handles = True)
    doc = tweet_token.tokenize(doc)
    #Gets rid of any tokens that represent if the tweet was retweeted
    doc = [w for w in doc if w.lower() not in common_tweet_words]
    #Gets rid  of any punctuation that we don't want to keep
    doc = [w for w in doc if w not in punct]
    #Lemmatizes tokens
    doc = pos_tag(doc)
    doc = [(w[0], pos_replace(w[1])) for w in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    
    return doc

def clean_tweets(doc):
    #Gets rid of links
    doc = re.sub(r'http\S+', 'http', doc)
    doc = re.sub(r'www\.[a-z]?\.?(com)+|[a-z]+\.(com)', 'http', doc)
    #Gets rid of #sxsw hashtag variations
    doc = re.sub(r'(?i)(#sxsw)\w*', '', doc)
    #Gets rid of conversions made during scraping
    doc = re.sub(r'{link}', '', doc)
    doc = re.sub(r'\[video\]', '', doc)
    #Gets rid of @'s associated with a RT
    doc = re.sub(r'RT @\w+', '', doc)
    #Gets rid of weird characters
    doc = re.sub(r'&quot;', '"', doc)
    doc = remove_junk(doc)
    return doc

In [3]:
tweets.tweet_text = tweets.tweet_text.apply(clean_tweets)

In [31]:
from transformers import pipeline

sentiment_pipeline = pipeline(model = "cardiffnlp/twitter-roberta-base-sentiment")

Generate predictions based on pre-trained model with no tuning

In [39]:
prediction = tweets.tweet_text.map(lambda x: sentiment_pipeline(x))

In [52]:
results = []

for i in range(len(tweets.tweet_text)):
    if prediction[i][0]['label'][-1] == '1':
        results.append(2) #Neutral
    elif prediction[i][0]['label'][-1] == '2':
        results.append(1) #Positive
    else:
        results.append(0) #Negative

In [55]:
from sklearn.metrics import accuracy_score

accuracy_score(tweets.label, results)

0.566138540899042

For being untuned it does pretty good! The final original model has 62% accurracy, so a result of 57% isn't bad. Ideally taking this model as a based and tuning it to the dataset would output better results.

# Fine Tuned Transformer

Preprocess the text and tokenize it properly

In [44]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

def preprocess(tweet):
    new_tweet = []
    for t in tweet.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_tweet.append(t)
    tweet_clean = " ".join(new_tweet)
    return tweet_clean.strip()

tweets_only = tweets[['tweet_text', 'label']]
tweets_only.tweet_text = tweets_only.tweet_text.apply(preprocess)

def tokenize_function(examples):
    return tokenizer(examples['tweet_text'], return_tensors='pt')

df = Dataset.from_pandas(tweets_only)
tokenized_datasets = df.map(tokenize_function)
split_df = train_test_split(tokenized_datasets)

0ex [00:00, ?ex/s]

In [51]:
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels = 3)

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
  
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

loading configuration file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/config.json from cache at /Users/kelseylane/.cache/huggingface/transformers/7dd97280b5338fb674b5372829a05a1aaaa76f9f2fa71c36199f2ce1ee1104a0.4c7ca95b4fd82b8bbe94fde253f5f82e5a4eedefe6f86f6fa79efc903d6cfe60
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_at

In [52]:
from transformers import TrainingArguments, Trainer
 
training_args = TrainingArguments("test_trainer")
 
trainer = Trainer(model = model, 
                  args = training_args, 
                  train_dataset = split_df[0], 
                  eval_dataset = split_df[1],
                 compute_metrics=compute_metrics)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
